In [12]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import model_from_yaml

os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [2]:
path = "../img/detection"
path_dog = os.path.join(path, 'dog')
path_nodog = os.path.join(path, 'no_dog')

path_dog_corgi = os.path.join(path_dog, 'corgi')
path_dog_shiba = os.path.join(path_dog, 'shiba')
path_dog_gr = os.path.join(path_dog, 'gr')

path_nodog_parrot = os.path.join(path_nodog, 'parrot')
path_nodog_tiger = os.path.join(path_nodog, 'tiger')
path_nodog_rabbit = os.path.join(path_nodog, 'rabbit')
class_names = ['dog', 'no_dog']

In [3]:
num_corgi = len(os.listdir(path_dog_corgi))
num_shiba = len(os.listdir(path_dog_shiba))
num_gr = len(os.listdir(path_dog_gr))
num_dogs = num_gr + num_shiba + num_corgi

num_parrot = len(os.listdir(path_nodog_parrot))
num_tiger = len(os.listdir(path_nodog_tiger))
num_rabbit = len(os.listdir(path_nodog_rabbit))
num_nodogs = num_parrot + num_tiger + num_rabbit

total_img_num = num_dogs + num_nodogs

print('total dog images:', num_dogs)
print('total no_dog images:', num_nodogs)
print('total images:', total_img_num)

total dog images: 3321
total no_dog images: 3199
total images: 6520


In [4]:
epochs = 26
IMG_HEIGHT = 112
IMG_WIDTH = 112
batch_size = 128

In [86]:
data_generator = ImageDataGenerator(rescale=1./255)

In [87]:
data_gen = data_generator.flow_from_directory(batch_size=batch_size,
                                              directory=path,
                                              shuffle=True,
                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                              class_mode='binary')

Found 6520 images belonging to 2 classes.


In [98]:
# all labels
labels = data_gen.classes

[0 0 0 ... 1 1 1]


In [89]:
## check encoded label
labels = (data_gen.class_indices)
print(labels)

{'dog': 0, 'no_dog': 1}
{0: 'dog', 1: 'no_dog'}


In [8]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.8),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.8),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
history = model.fit_generator(
    data_gen,
    steps_per_epoch=total_img_num // batch_size,
    epochs=epochs
)

Epoch 1/26
50/50 [==============================] - 27s 547ms/step - loss: 0.3297 - acc: 0.8614
Epoch 2/26
50/50 [==============================] - 27s 536ms/step - loss: 0.3335 - acc: 0.8622
Epoch 3/26
50/50 [==============================] - 27s 531ms/step - loss: 0.3232 - acc: 0.8633
Epoch 4/26
50/50 [==============================] - 28s 552ms/step - loss: 0.3228 - acc: 0.8651
Epoch 5/26
50/50 [==============================] - 28s 552ms/step - loss: 0.2973 - acc: 0.8764
Epoch 6/26
50/50 [==============================] - 29s 573ms/step - loss: 0.2871 - acc: 0.8839
Epoch 7/26
50/50 [==============================] - 28s 551ms/step - loss: 0.2862 - acc: 0.8864
Epoch 8/26
50/50 [==============================] - 27s 544ms/step - loss: 0.2701 - acc: 0.8908
Epoch 9/26
50/50 [==============================] - 27s 543ms/step - loss: 0.2767 - acc: 0.8828
Epoch 10/26
50/50 [==============================] - 26s 524ms/step - loss: 0.2717 - acc: 0.8820
Epoch 11/26
50/50 [====================

## Save

In [13]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

# serialize weights to HDF5
model.save_weights("model.h5")

## Load

In [14]:
# load YAML and create model
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)

# load weights into new model
loaded_model.load_weights("model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\ABC\Anaconda3\lib\site-packages\tensorflow_core\python\keras\saving\model_config.py:76: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_string)


In [76]:
loaded_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [77]:
predict_img = "../img/test_img"

predict_gen = data_generator.flow_from_directory(directory=predict_img,
                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                              class_mode='binary',
                                                 shuffle=False)

Found 4 images belonging to 1 classes.


In [90]:
predict = loaded_model.predict_generator(predict_gen)
print(predict)

[[0.02821344]
 [0.04785806]
 [0.11119083]
 [0.8936862 ]]


In [93]:
## get label
y_pred = np.rint(predict)
print(y_pred)

[[0.]
 [0.]
 [0.]
 [1.]]


In [ ]:
## for multiclass
predicted_class_indices = np.argmax(pred, axis=1)
predictions = [label[i] for i in predicted_class_indices]

In [85]:
## check filename
filenames = predict_gen.filenames
print(filenames)

['test\\1.jpg', 'test\\2.jpg', 'test\\3.jpg', 'test\\4.jpg']
